In [9]:
import random
import numpy as np
import time
import pandas as pd
from os.path import expanduser

In [2]:
import pathlib
from pprint import pprint

import opentimspy
from opentimspy.opentims import OpenTIMS

In [3]:
path = pathlib.Path('/media/big-ssd/experiments/P3856/raw-databases/P3856_YHE211_1_Slot1-1_1_5104.d')
D = OpenTIMS(path) # get data handle

#### loading all the ms1 frames

In [4]:
start_run = time.time()

df_l = []
for idx,d in enumerate(D.query_iter(D.ms1_frames, columns=('frame','mz','scan','intensity','retention_time'))):
    d['frame'] = d['frame'].astype(np.uint16, copy=False)
    d['mz'] = d['mz'].astype(np.float64, copy=False)
    d['scan'] = d['scan'].astype(np.uint16, copy=False)
    d['intensity'] = d['intensity'].astype(np.uint16, copy=False)
    d['retention_time'] = d['retention_time'].astype(np.float64, copy=False)
    df_l.append(pd.DataFrame(d))
df = pd.concat(df_l, axis=0, sort=False, ignore_index=True)
# df = df[(df.mz >= 780) & (df.mz <= 795)]

stop_run = time.time()
print("loaded {} points in {} seconds".format(len(df), round(stop_run-start_run,1)))

loaded 1255020569 points in 91.7 seconds


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255020569 entries, 0 to 1255020568
Data columns (total 5 columns):
 #   Column          Dtype  
---  ------          -----  
 0   frame           uint16 
 1   mz              float64
 2   scan            uint16 
 3   intensity       uint16 
 4   retention_time  float64
dtypes: float64(2), uint16(3)
memory usage: 25.7 GB


In [6]:
(df.retention_time.max()-df.retention_time.min())/60

48.01525684836269

In [10]:
df.to_feather('{}/P3856_YHE211_1_Slot1-1_1_5104-ms1.feather'.format(expanduser('~')))

#### loading all the ms2 frames

In [7]:
ms2_frame_ids = np.arange(D.min_frame, D.max_frame+1)[D.ms_types == 8]

In [8]:
ms2_df_l = []
for idx,d in enumerate(D.query_iter(ms2_frame_ids, columns=('frame','mz','scan','intensity','retention_time'))):
    d['frame'] = d['frame'].astype(np.uint16, copy=False)
    d['mz'] = d['mz'].astype(np.float64, copy=False)
    d['scan'] = d['scan'].astype(np.uint16, copy=False)
    d['intensity'] = d['intensity'].astype(np.uint16, copy=False)
    d['retention_time'] = d['retention_time'].astype(np.float64, copy=False)
    ms2_df_l.append(pd.DataFrame(d))
ms2_df = pd.concat(ms2_df_l, axis=0, sort=False, ignore_index=True)

In [9]:
len(ms2_df)

37253651

#### loading a 3D segment

load all the data as above, and discard the m/z and scan ranges not needed.